# Restaurant Recommendation System
## Theo Jeremiah

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Import Data

In [ ]:
df = pd.read_csv('yelp_review_arizona.csv')

In [4]:
df = yelp[['review_id', 'user_id', 'business_id', 'text', 'stars', 'date']]

In [5]:
df_business = pd.read_csv('yelp_business.csv')

In [6]:
#Check Null values in Dataframe
df.isnull().sum()

review_id      0
user_id        0
business_id    0
text           0
stars          0
date           0
dtype: int64

In [7]:
df.head()

,review_id,user_id,business_id,text,stars,date
0,V93SYj2OLh5m9Cquzf-7kg,ZwVz20be-hOZnyAbevyMyQ,2c9Vptks_vowLgVUMnCgjw,Came here while in town for a country concert....,4.0,2013-09-04 01:29:46
1,vNTFadc6T9HeH3Qa78dc_Q,91TB-gzcNyxFh46TL0pmnQ,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue this side of the Mississippi!!!!...,5.0,2015-12-05 02:50:10
2,SXRFBCt5eXCBF7TlI7UG6Q,Y_QBiZpATJoz8hKUfYF66A,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing. Think Chipotle for enchila...,5.0,2014-04-01 01:56:00
3,CqMNjtG0hNZGhDw4RDE-zw,_Jg-IA0M-GSjBlGu-wmejg,r8764MtYyt8JhxMvrfM_xQ,I was really disappointed with my most recent ...,2.0,2014-10-11 03:53:53
4,5hZLouGEW4wm6BTJ5aNUNw,1CqkFliipv_X15WYn5aPfg,QS3QxI7u5PRdtbGgI0-UsA,I grade sushi restaurants on 3 factors:\n- Qua...,4.0,2015-03-04 19:36:21


In [8]:
df.shape

(26606, 6)

### Select only stars and text

In [9]:
#Select only stars and text
yelp_data = df[['business_id', 'user_id', 'stars', 'text']]

In [10]:
import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [11]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

### Clean Text

In [12]:
yelp_data['text'] = yelp_data['text'].apply(text_process)

In [13]:
#Split train test for testing the model later
vld_size=0.15
X_train, X_valid, y_train, y_valid = train_test_split(yelp_data['text'], df['business_id'], test_size = vld_size) 

### Create two tables of user,text and business,text

In [14]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [15]:
userid_df.head()

,user_id,text
0,ZwVz20be-hOZnyAbevyMyQ,Came town country concert better way start day...
1,91TB-gzcNyxFh46TL0pmnQ,Best barbecue side Mississippi Come car washed...
2,Y_QBiZpATJoz8hKUfYF66A,Absolutely amazing Think Chipotle enchiladas N...
3,_Jg-IA0M-GSjBlGu-wmejg,really disappointed recent visit Vintage 95 ma...
4,1CqkFliipv_X15WYn5aPfg,grade sushi restaurants 3 factors Quality Exce...


In [16]:
userid_df[userid_df['user_id']=='ZwVz20be-hOZnyAbevyMyQ']['text']

0        Came town country concert better way start day...
1098     Amazing Im golfer ended loving went family lov...
3909     saw many great reviews place decided try high ...
11491    Amazing Mexican Asian infusion good service sa...
19647    beautiful hotel excellent service Nick front a...
Name: text, dtype: object

In [17]:
business_df.head()

,business_id,text
0,2c9Vptks_vowLgVUMnCgjw,Came town country concert better way start day...
1,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue side Mississippi Come car washed...
2,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing Think Chipotle enchiladas N...
3,r8764MtYyt8JhxMvrfM_xQ,really disappointed recent visit Vintage 95 ma...
4,QS3QxI7u5PRdtbGgI0-UsA,grade sushi restaurants 3 factors Quality Exce...


### Join the text with for each user_id and business_id

In [18]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [19]:
userid_df.head()

,text
user_id,
--2HUmLkcNHZp0xw6AMBPg,place JAM Surfer vibe great eats love machaca ...
--4rAAfZnEIAKJE80aIiYg,pulled pork spicy bbq sauce impressed Probably...
--Nnm_506G_p8MxAOQna5w,Cant say burger anything special Taste ok Shak...
--ty7Z9fEt08E3dS3_qoSA,know think important trust Yelp kind reviews t...
-0IiMAZI2SsQ7VmyzJjokQ,Ever fan roadside attractions Americana Id rea...


In [20]:
userid_df.loc['ZwVz20be-hOZnyAbevyMyQ']['text']

'Came town country concert better way start day Everything great service amazing time walked door sat table surprise beer whiskey tap table pay ounce screen show drank food great whiskey burger favorite country fans must Scottsdale Amazing Im golfer ended loving went family loves golf planned staying hour since golf 6 hours later left far drive place right food amazing expect good brisket tacos chorizo sliders favorite Good quality food many unique items highly recommend place people ages teens loved bays nice roomy 7 us bay 10 people must try back soon saw many great reviews place decided try high hopes loved atmosphere kids loved Texas trivia game thats good thing could say steak came cooked way asked took back fix steak flavor covered butter thought steak even kids eat theres waitress asked daughter something wrong since ate bites explained like said ok walked away shrimp good pricey bugers ok end pushing us take home since ate almost none politely declined still came boxes trying g

## User Tfidf Vectorizer with 5000 Features (represent 88% of all words)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
userid_vectors.shape

(10937, 5000)

In [23]:
userid_vectors

<10937x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1083887 stored elements in Compressed Sparse Row format>

In [24]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
businessid_vectors.shape

(1411, 5000)

# Matrix Factorization

In [25]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(10937, 1411)

In [26]:
userid_rating_matrix.head()

business_id,-050d_XIor1NpCuWkbIVaQ,-1UMR00eXtwaeh59pEiDjA,-4TMQnQJW1yd6NqGRDvAeA,-6h3K1hj0d4DRcZNUtHDuw,-8QlV3b_9H4BAh6LgMIr1g,-9eNGMp8XiygI8t8QFuFWw,-9nai28tnoylwViuJVrYEQ,-Bdw-5H5C4AYSMGnAvmnzw,-BxWyEIQ6wypT-37MzZizQ,-CfFjcCcGGDM9MVH_d42RQ,...,z_lDO8d8nkSmcvTjB4N69A,za9qr9ZZWLTfEgTfogRbUw,zbrFk-4ejesAJD8EwcdHxg,zfiSQ1dl3vTJ-og96eqXGA,zidkKI_N1OPxsiddTOQH_Q,zqNgwQjj0_XAll-neGikIw,zr93wrNyXzc-HW4IcK4iRQ,zrDi4gEaUi64lAMfJU51dw,zrTGcb83AsfyVTMrsCa65A,zuVvDYJkKAbXQTTBauAqJQ
user_id,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--4rAAfZnEIAKJE80aIiYg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Nnm_506G_p8MxAOQna5w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ty7Z9fEt08E3dS3_qoSA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0IiMAZI2SsQ7VmyzJjokQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [28]:
Q.head()

,0,1,10,100,1000,101,1015,1030,10pm,11,...,z,zero,zest,zinburger,zinc,zipps,ziti,zookz,ztejas,zucchini
business_id,,,,,,,,,,,,,,,,,,,,,
-050d_XIor1NpCuWkbIVaQ,0.0,0.008223,0.007567,0.018716,0.0,0.0,0.0,0.01094,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1UMR00eXtwaeh59pEiDjA,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-4TMQnQJW1yd6NqGRDvAeA,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-6h3K1hj0d4DRcZNUtHDuw,0.0,0.028893,0.026588,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-8QlV3b_9H4BAh6LgMIr1g,0.0,0.000000,0.017147,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Gradient Decent Optimization

In [29]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q

In [30]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

Wall time: 4h 34min 22s


In [42]:
Q.head()

,0,1,10,100,1000,101,1015,1030,10pm,11,...,z,zero,zest,zinburger,zinc,zipps,ziti,zookz,ztejas,zucchini
business_id,,,,,,,,,,,,,,,,,,,,,
-050d_XIor1NpCuWkbIVaQ,0.001158,0.046774,0.043362,0.041756,0.006478,0.002138,0.010029,0.023638,0.000542,0.001409,...,0.000139,0.000476,0.000238,0.013895,0.000119,0.000197,0.001806,0.000067,0.000085,0.004687
-1UMR00eXtwaeh59pEiDjA,0.000126,0.001169,0.001863,0.000566,0.000278,0.000095,0.000099,0.000047,0.000095,0.000339,...,0.000003,0.000444,0.000181,0.000015,0.000010,0.000197,0.000387,0.000018,0.000008,0.000711
-4TMQnQJW1yd6NqGRDvAeA,0.000497,0.004620,0.003707,0.001494,0.000228,0.000101,0.000558,0.000692,0.000350,0.001170,...,0.000332,0.000700,0.000394,0.000153,0.000077,0.004837,0.000070,0.000053,0.001040,0.000478
-6h3K1hj0d4DRcZNUtHDuw,0.000447,0.050076,0.039643,0.001207,0.000276,0.001007,0.000561,0.000198,0.000451,0.000763,...,0.002316,0.002917,0.000074,0.000897,0.000926,0.000124,0.000099,0.000040,0.007819,0.001039
-8QlV3b_9H4BAh6LgMIr1g,0.000031,0.027002,0.043027,0.001043,0.000206,0.000046,0.000047,0.000084,0.000195,0.018048,...,0.000058,0.006484,0.000131,0.000024,0.000069,0.000160,0.000211,0.000009,0.000066,0.000084


In [46]:
Q.iloc[0].sort_values(ascending=False).head(10)

matts        1.077482
breakfast    0.659543
wait         0.529304
food         0.428016
place        0.405557
eggs         0.362205
good         0.354656
bacon        0.312531
like         0.274301
big          0.265078
Name: -050d_XIor1NpCuWkbIVaQ, dtype: float64

### Export Model

In [31]:
# Store P, Q and vectorizer in pickle file
import pickle
output = open('yelp_recommendation_model_8.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()

## Prediction for input text

In [32]:
words = "i want to have dinner with beautiful views"
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]

for i in topRecommendations.index:
    print(df_business[df_business['business_id']==i]['name'].iloc[0])
    print(df_business[df_business['business_id']==i]['categories'].iloc[0])
    print(str(df_business[df_business['business_id']==i]['stars'].iloc[0])+ ' '+str(df_business[df_business['business_id']==i]['review_count'].iloc[0]))
    print('')


Isabella's Kitchen
Bars, Nightlife, Beer, Wine & Spirits, Italian, Food, Restaurants
3.5 259

Compass Arizona Grill
Nightlife, American (New), Bars, Restaurants, Lounges
3.5 540

Chart House
Event Planning & Services, Restaurants, Venues & Event Spaces, American (Traditional), Seafood, Nightlife, Bars, Steakhouses, Wine Bars
3.0 229

Different Pointe of View
Restaurants, American (Traditional), Italian, Mediterranean, Resorts, Hotels & Travel
4.0 512

Orange Sky
Seafood, Steakhouses, American (New), Restaurants
4.0 562

Steak 44
Restaurants, Steakhouses, Bars, Nightlife, Wine Bars, Seafood
4.5 950

Top of the Rock Restaurant
Restaurants, American (New), Hotels & Travel, Event Planning & Services, Hotels
3.5 325

